# Processing of meterological data

In [5]:
import numpy as np
import pandas as pd

# Functions

In [6]:
def Gumbel_for_one(series_of_values, name):
    """Calculation of Gumbel distribution for any characteristics.
          
        series_of_values: pd.series, import values
        
        name: str, name of characteristic
    """
    
    table1 = pd.read_csv('table_of_n_y.csv', delimiter = ';')
    table2 = pd.read_csv('table_of_n_sigma.csv', delimiter = ';')

    from scipy import interpolate
    inter_y = interpolate.interp1d(table1['n'], table1['y_mean'])
    inter_sigma = interpolate.interp1d(table2['n'], table2['sigma'])
    
    # 
    data = series_of_values
    
    MSE = data.std()
    mean = data.mean()
    y = inter_y(len(data))
    sigma = inter_sigma(len(data))
    q = mean - y * (MSE / sigma)

    p_values = [0.01, 0.1, 1, 5, 10, 20, 30, 50, 63, 70, 80, 90, 95, 99, 99.9]
    
    p_df = pd.DataFrame(data=p_values, columns=['P_%'])

    from numpy import log as ln
    p_df['Yp'] = -ln(-ln((100 - p_df['P_%']) / 100))
    p_df[name] = q + (MSE / sigma) * p_df['Yp']

    p_df[name] = p_df[name].apply(lambda x: 0 if x < 0 else x)
    p_df = p_df.drop('Yp', axis=1)
    p_df.to_excel('{}.xlsx'.format(name), float_format='%.2f')
    
    return p_df.round(1)

In [80]:
def year_mean_distribution (df, I, start_year, end_year):
    """ Calculates mean value of characteristic for months and year
        
        df: dataframe, contains columns ['Year', 'Month', I]
        
        I: str, name of characteristic
        
        start_year: year from which the calculation begins
        
        end_year: year which ends the calculation"""
    
    import numpy as np
    import pandas as pd
    
    time_names =  [1,2,3,4,5,6,7,8,9,10,11,12,'Год']
    
    df = df[['Year', 'Month', I]].dropna()
    df = df[df['Year'].between(start_year, end_year)]
    df_piv = df.pivot_table(columns='Month', index='Year', aggfunc=np.mean)
    df_piv = df_piv.dropna(axis=0)
    df_piv['year_value'] = df_piv.mean(axis=1)
    df_year = df_piv.mean(axis=0).to_frame().droplevel(0, axis=0)
    df_year.columns = [I]
    df_year = df_year.reset_index().drop('Month', axis=1)

    result = df_year.round(1).transpose()
    result.columns = time_names
    result.to_excel('{}.xlsx'.format(I), index=True)
    
#     from plotly.offline import init_notebook_mode, iplot
#     import plotly
#     import plotly.graph_objs as go
#     init_notebook_mode(connected=True)
    
#     trace0 = go.Scatter(
#     x = df_year.index,
#     y = df_year.iloc[:,0].round(1),
#     name = I)
    
#     data = [trace0]
#     layout = {'title': 'Year distribution of {}'.format(I)}
#     fig = go.Figure(data=data, layout=layout)
#     iplot(fig, show_link=False)
      
    return result

In [8]:
    def imper_table(df, char):
        """Calculates empirical probability values for char.
          Need a function 'save_as_excel_table'

            df: dataframe, contains columns ['Year', char]

            char: str, name of characteristic"""
    
    
        df.columns = ['Year', char]
        df = df.sort_values(by=char, ascending=False).reset_index().drop('index', axis=1)
        df['length'] = range(1, len(df)+1)
        df['imper_probab_%'] = df['length'] / (len(df) + 1) * 100 
        df['imper_probab_%'] = df['imper_probab_%'].round(3)


        df = df[['length', 'Year', char, 'imper_probab_%']]
        df.columns = ['№', 'Год', 'Значение', 'Обеспеченность P(%)']

        save_as_excel_table('imper_table_of_{}'.format(char), 1 , df, '00')

In [9]:
def save_as_excel_table(excel_file_name, sheet_name, df, float_num):
    """Save dataframe as xlsx file
        excel_file_name: tr,wihtout .xlsx
        
        sheet_name: any, of sheet in file
        
        df: dataframe, ta
        
        float_num: numbers of float nulls. In format - '00' - means two zeros"""
    
    from styleframe import StyleFrame, Styler, utils
    import openpyxl
    
    with StyleFrame.ExcelWriter('{}.xlsx'.format(excel_file_name)) as writer:
        
        sf=StyleFrame(df)
        
        # table
        sf.apply_column_style(cols_to_style=df.columns, \
                              styler_obj=Styler(bg_color=utils.colors.white, \
                                                bold=False, \
                                                font=utils.fonts.calibri,\
                                                font_size=11),
                              style_header=False)
        
        # header
        sf.apply_headers_style(styler_obj=Styler(bg_color=utils.colors.white, \
                                                 bold=True, \
                                                 font_size=11, \
                                                 font_color=utils.colors.black,\
                                                 number_format=utils.number_formats.general, \
                                                 protection=False))
        
        
        sf.to_excel(writer, sheet_name=str(sheet_name), index = False)
        writer.save()
        
    import openpyxl  
    wb = openpyxl.load_workbook(filename = '{}.xlsx'.format(excel_file_name))
    ws = wb['{}'.format(str(sheet_name))]
    
    
    for x in range(1,51):
        for y in range(1,51):
            ws.cell(row=x, column=y).number_format = '0.{}'.format(float_num)
    wb.save('{}.xlsx'.format(excel_file_name))
             
        
    return(print('result in file "{}.xlsx"'.format(excel_file_name)))

# Data

In [10]:
df = pd.read_csv('sroki_1.csv', delimiter=';')

In [11]:
df

,ID,Year,Month,Day,Time,Wind_dir,Wind_vel_mean,Wind_vel_max,Humidity,Pressure_st,Pressure_sea
0,30965,1966,1,1,18,70.0,3.0,NaN,74.0,941.2,NaN
1,30965,1966,1,1,21,70.0,3.0,NaN,74.0,941.1,NaN
2,30965,1966,1,1,0,70.0,3.0,NaN,74.0,941.6,NaN
3,30965,1966,1,1,3,90.0,1.0,NaN,75.0,942.5,NaN
4,30965,1966,1,1,6,0.0,0.0,NaN,74.0,943.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...
158027,30965,2020,1,31,0,31.0,1.0,2.0,74.0,937.8,1029.1
158028,30965,2020,1,31,3,206.0,1.0,3.0,75.0,938.4,1027.8
158029,30965,2020,1,31,6,282.0,3.0,5.0,67.0,937.8,1024.9
158030,30965,2020,1,31,9,0.0,0.0,5.0,72.0,938.8,1026.2


# Wind
## Maximum velocity

In [12]:
wind_max = df[['Year', 'Month', 'Day', 'Time', 'Wind_vel_max']]
wind_max.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158032 entries, 0 to 158031
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Year          158032 non-null  int64  
 1   Month         158032 non-null  int64  
 2   Day           158032 non-null  int64  
 3   Time          158032 non-null  int64  
 4   Wind_vel_max  125885 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.0 MB


In [13]:
wind_max.isnull().sum()

Year                0
Month               0
Day                 0
Time                0
Wind_vel_max    32147
dtype: int64

In [14]:
wind_max = wind_max.dropna(axis=0)
wind_max = wind_max[wind_max['Year'].between(1977, 2019)]

In [15]:
wind_max.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125637 entries, 32144 to 157783
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Year          125637 non-null  int64  
 1   Month         125637 non-null  int64  
 2   Day           125637 non-null  int64  
 3   Time          125637 non-null  int64  
 4   Wind_vel_max  125637 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 5.8 MB


In [16]:
index_to_drop = wind_max[wind_max['Wind_vel_max']==0].index
wind_max_dropped = wind_max.drop(index_to_drop, axis=0)
wind_max_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123609 entries, 32144 to 157783
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Year          123609 non-null  int64  
 1   Month         123609 non-null  int64  
 2   Day           123609 non-null  int64  
 3   Time          123609 non-null  int64  
 4   Wind_vel_max  123609 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 5.7 MB


In [17]:
wind_maximum = wind_max_dropped.groupby('Year')['Wind_vel_max'].max()
wind_maximum 

Year
1977    26.0
1978    20.0
1979    24.0
1980    24.0
1981    34.0
1982    27.0
1983    28.0
1984    29.0
1985    29.0
1986    24.0
1987    24.0
1988    26.0
1989    25.0
1990    27.0
1991    23.0
1992    23.0
1993    24.0
1994    24.0
1995    26.0
1996    24.0
1997    24.0
1998    24.0
1999    28.0
2000    24.0
2001    27.0
2002    20.0
2003    34.0
2004    31.0
2005    23.0
2006    20.0
2007    23.0
2008    28.0
2009    27.0
2010    21.0
2011    24.0
2012    24.0
2013    23.0
2014    20.0
2015    20.0
2016    34.0
2017    27.0
2018    25.0
2019    24.0
Name: Wind_vel_max, dtype: float64

In [19]:
Gumbel_for_one(wind_maximum, 'wind_max')

,P_%,wind_max
0,0.0,52.1
1,0.1,45.0
2,1.0,37.8
3,5.0,32.8
4,10.0,30.5
5,20.0,28.2
6,30.0,26.8
7,50.0,24.7
8,63.0,23.6
9,70.0,23.0


In [20]:
imper_table(wind_maximum.to_frame().reset_index(), 'wind_max')

result in file "imper_table_of_wind_max.xlsx"


## Average velocity

In [21]:
wind_av = df[['Year', 'Month', 'Day', 'Time', 'Wind_vel_mean']]
wind_av.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158032 entries, 0 to 158031
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Year           158032 non-null  int64  
 1   Month          158032 non-null  int64  
 2   Day            158032 non-null  int64  
 3   Time           158032 non-null  int64  
 4   Wind_vel_mean  158024 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.0 MB


In [22]:
wand_av = wind_av.dropna(axis=0)
wand_av.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158024 entries, 0 to 158031
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Year           158024 non-null  int64  
 1   Month          158024 non-null  int64  
 2   Day            158024 non-null  int64  
 3   Time           158024 non-null  int64  
 4   Wind_vel_mean  158024 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 7.2 MB


In [81]:
year_mean_distribution (wind_av, 'Wind_vel_mean', 1966, 2019)

,1,2,3,4,5,6,7,8,9,10,11,12,Год
Wind_vel_mean,1.8,2.2,3.2,4.3,4.3,3.3,2.9,2.8,3.1,3.1,2.4,1.8,2.9


In [24]:
Gumbel_for_one(wind_av.groupby('Year')['Wind_vel_mean'].mean(), 'wind_av_vel_Gumbel')

,P_%,wind_av_vel_Gumbel
0,0.0,5.7
1,0.1,4.9
2,1.0,4.2
3,5.0,3.7
4,10.0,3.4
5,20.0,3.2
6,30.0,3.1
7,50.0,2.8
8,63.0,2.7
9,70.0,2.7


In [25]:
df_av_imp = wind_av.groupby('Year')['Wind_vel_mean'].mean().to_frame().reset_index()
imper_table(df_av_imp, 'Wind_vel_mean')

result in file "imper_table_of_Wind_vel_mean.xlsx"


## Repeability

In [26]:
wind_rep = df[['Year', 'Month', 'Day', 'Time', 'Wind_dir']]
wind_rep = wind_rep[wind_rep['Year'].between(1977, 2019)]

In [27]:
wind_rep = wind_rep.dropna(axis=0)
wind_rep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125637 entries, 32144 to 157783
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Year      125637 non-null  int64  
 1   Month     125637 non-null  int64  
 2   Day       125637 non-null  int64  
 3   Time      125637 non-null  int64  
 4   Wind_dir  125637 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 5.8 MB


In [28]:
wind_rep['Wind_dir'].astype('int64')

32144       0
32145      70
32146       0
32147      70
32148       0
         ... 
157779    215
157780    280
157781    258
157782    218
157783    203
Name: Wind_dir, Length: 125637, dtype: int64

In [29]:
# cardinal points
N1 = range(1, 23)
NE = range(23, 68)
E = range(68, 113)
SE = range(113, 158)
S = range(158, 203)
SW = range(203, 248)
W = range(248, 293) 
NW = range(293, 338)
N2 = range(338, 361)
calm = [0]

CP = [N1, NE, E, SE, S, SW, W, NW, N2, calm]
CP

[range(1, 23),
 range(23, 68),
 range(68, 113),
 range(113, 158),
 range(158, 203),
 range(203, 248),
 range(248, 293),
 range(293, 338),
 range(338, 361),
 [0]]

In [30]:
# cardinal_points = {'N1': range(0, 22), 'NE': range(23, 67), 'E': range(68, 112), 'SE': range(113, 157), 'S': range(158, 202),
#                    'SW': range(203, 247), 'W': range(248, 292) , 'NW':range(293, 337), 'N2': range(338, 360), 'calm': 0}

In [31]:
#type(list(cardinal_points.values()))

In [44]:
wind_rep
len(wind_rep['Wind_dir'].astype('int64'))
wind_rep['Wind_dir'] = wind_rep['Wind_dir'].astype('int64')
wind_rep

,Year,Month,Day,Time,Wind_dir
32144,1977,1,1,18,0
32145,1977,1,1,21,70
32146,1977,1,1,0,0
32147,1977,1,1,3,70
32148,1977,1,1,6,0
...,...,...,...,...,...
157779,2019,12,31,0,215
157780,2019,12,31,3,280
157781,2019,12,31,6,258
157782,2019,12,31,9,218


In [57]:
direction = []

for wind in wind_rep['Wind_dir']:
    
    for n in CP:
        
        for degree in n:
            
            if wind == degree:
                direction.append(n)

In [58]:
len(direction)

125637

In [59]:
cardinal_points = {range(1, 23): 'N1', range(23, 68):'NE', range(68, 113): 'E', range(113, 158) : 'SE', range(158, 203): 'S',
                  range(203, 248):  'SW', range(248, 293): 'W', range(293, 338): 'NW', range(338, 361): 'N2', 0: 'calm'}

In [60]:
cardinal_points.get((0))

'calm'

In [61]:
direction_get = []

for ran in direction:
    
    if ran == [0]:
        ran = 0
                
    name_of_CP = cardinal_points.get(ran)
    
    direction_get.append(name_of_CP)

In [62]:
len(direction_get)

125637

In [63]:
wind_rep['Direction'] = direction_get

In [64]:
wind_rep.head(10)

,Year,Month,Day,Time,Wind_dir,Direction
32144,1977,1,1,18,0,calm
32145,1977,1,1,21,70,E
32146,1977,1,1,0,0,calm
32147,1977,1,1,3,70,E
32148,1977,1,1,6,0,calm
32149,1977,1,1,9,220,SW
32150,1977,1,1,12,250,W
32151,1977,1,1,15,0,calm
32152,1977,1,2,18,0,calm
32153,1977,1,2,21,0,calm


### Year

In [65]:
wind_rep['Direction'].value_counts().to_excel('year-direction.xlsx')

### Year values without calm

In [79]:
wind_rep_2 = wind_rep[wind_rep['Direction'] != 'calm']
wind_rep_2['Direction'].value_counts().to_excel('wind_year_without calm.xlsx')

### Calm for months

In [67]:
wind_rep_calm = wind_rep[wind_rep['Direction'] == 'calm']
wind_rep_calm.groupby('Month')['Direction'].value_counts().to_excel('wind_calm.xlsx')

### Months with calm

In [68]:
wind_rep.groupby('Month')['Direction'].value_counts().to_excel('wind_direction.xlsx')

In [69]:
wind_rep_3 = wind_rep[wind_rep['Direction'] == 'calm']

In [70]:
wind_rep_3
wind_rep_3['Direction'].value_counts()

calm    17651
Name: Direction, dtype: int64

### Months without calm

In [71]:
mask = wind_rep['Direction'] != 'calm'
wind_rep[mask].groupby('Month')['Direction'].value_counts().to_excel('wind_dir_without_calm.xlsx')

## Humidity

In [72]:
humidity = df[['Year', 'Month', 'Day', 'Time', 'Humidity']]
humidity

,Year,Month,Day,Time,Humidity
0,1966,1,1,18,74.0
1,1966,1,1,21,74.0
2,1966,1,1,0,74.0
3,1966,1,1,3,75.0
4,1966,1,1,6,74.0
...,...,...,...,...,...
158027,2020,1,31,0,74.0
158028,2020,1,31,3,75.0
158029,2020,1,31,6,67.0
158030,2020,1,31,9,72.0


In [73]:
humidity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158032 entries, 0 to 158031
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Year      158032 non-null  int64  
 1   Month     158032 non-null  int64  
 2   Day       158032 non-null  int64  
 3   Time      158032 non-null  int64  
 4   Humidity  158021 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.0 MB


In [74]:
humidity = humidity.dropna(axis=0)
humidity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158021 entries, 0 to 158031
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Year      158021 non-null  int64  
 1   Month     158021 non-null  int64  
 2   Day       158021 non-null  int64  
 3   Time      158021 non-null  int64  
 4   Humidity  158021 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 7.2 MB


### Cold period

In [75]:
hum_cold = humidity[(humidity['Month'].between(1,3)) | (humidity['Month'].between(11,12))]
hum_cold['Month'].value_counts()

1     13638
12    13392
3     13391
11    12958
2     12199
Name: Month, dtype: int64

In [76]:
hum_cold['Humidity'].mean()

74.42000365976395

### Warm period

In [77]:
hum_warm = humidity[humidity['Month'].between(4,10)]
hum_warm['Month'].value_counts()

8     13392
7     13392
5     13392
10    13389
6     12960
9     12959
4     12959
Name: Month, dtype: int64

In [78]:
hum_warm['Humidity'].mean()

60.93746416710838